In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 19.0 MB/s 
     |████████████████████████████████| 636 kB 30.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset
import yaml
import torch
import pandas as pd
import torch
from torch.utils.data import random_split
from transformers import TrainingArguments, Trainer, GPTNeoForCausalLM
from transformers import GPT2TokenizerFast as GPT2Tokenizer
import yaml
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
from os import listdir

In [ ]:
with open('/content/drive/MyDrive/data/config.yml', 'r') as f:
    config = yaml.safe_load(f)
torch.cuda.empty_cache()

tokenizer = GPT2Tokenizer.from_pretrained(config['model']['model_name_internacional'], eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained(config['model']['model_name_internacional'])
model.resize_token_embeddings(len(tokenizer))

"""Limpando Cache e Designando Variáveis Aleatórias"""
torch.cuda.empty_cache()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [ ]:
!nvidia-smi

Mon Aug 16 21:05:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
path = config['data']['path_internacional']

In [ ]:
for file in listdir(path):
  print(file)

panic-at-the-disco.json
my-chemical-romance.json
paramore.json
fall-out-boy.json
simple-plan.json
good-charlotte.json
three-days-grace.json
the-used.json
a-day-to-remember.json
sum-41.json
tokio-hotel.json
green-day.json
avril-lavigne.json
linkin-park.json
evanescence.json


In [ ]:
df = pd.DataFrame()
for file in listdir(path):
  temp = pd.read_json(path + r'/' + file)
  df = df.append(temp, ignore_index=True)

In [ ]:
def removendo_frases_repetidas(texto):
  seen = set()
  return [x for x in texto if x not in seen and not seen.add(x)]

In [ ]:
def removendo_sentencas_em_paratenses(x):
    return [re.sub(r'\([^)]*\)', '', y) for y in x]

In [ ]:
def removendo_sentencas_em_cerquilhas(x):
    return [re.sub(r'\{[^)]*\}', '', y) for y in x]

In [ ]:
def removendo_sentencas_em_colchetes(x):
    return [re.sub(r'\[[^)]*\]', '', y) for y in x]

In [ ]:
def removendo_linha_vazia(x):
  return list(filter(lambda x: x != '', x))

In [ ]:
def removendo_refrao(x):
  for y in x:
    return re.sub('[Cc]horus', '', y)

In [ ]:
def removendo_musica_sem_letra(x):
  if len(x) <= 3:
    return float('NaN')
  else:
    return x

In [ ]:
df['letra'] = df['letra'].apply(lambda x: removendo_frases_repetidas(x))
df['letra'] = df['letra'].apply(lambda x: removendo_sentencas_em_paratenses(x))
df['letra'] = df['letra'].apply(lambda x: removendo_sentencas_em_colchetes(x))
df['letra'] = df['letra'].apply(lambda x: removendo_sentencas_em_cerquilhas(x))
df['letra'] = df['letra'].apply(lambda x: removendo_linha_vazia(x))
df['letra'] = df['letra'].apply(lambda x: removendo_musica_sem_letra(x))
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
len(df)

2381

In [ ]:
#tornando letras em string
string = ''
for i, x in enumerate(df['letra']):
  for y in x:
    string += y + ' ' + '\n' + ' '
  x = string
  df['letra'][i] = x
  string = ''

In [ ]:
train, test = train_test_split(df['letra'], test_size=0.15, shuffle=True, random_state=SEED)

In [ ]:
class ShapingDataset(Dataset):

    def __init__(self, texts):
        self.texts = texts
        self.tokenizer = tokenizer
        self.labels = []
        self.input_ids = []
        self.attention_masks = []
        self.max_length = max([len(re.findall('\w+', x)) for x in df['letra']]) #letra com maior número de palavras
        for txt in texts:
            dici = self.tokenizer(txt, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
            self.input_ids.append(dici['input_ids'][0])
            self.attention_masks.append(dici['attention_mask'][0])

    def __len__(self):
        return len(self.texts)
  
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [ ]:
train = ShapingDataset(train)
test = ShapingDataset(test)

In [ ]:
training_args = TrainingArguments(output_dir=config['data']['output_dir'],
                                  logging_dir=config['data']['logging_dir_internacional'],
                                  num_train_epochs=config['model']['num_epochs'],
                                  per_device_train_batch_size=config['model']['batch_size'],
                                  per_device_eval_batch_size=config['model']['batch_size'],
                                  logging_steps=config['data']['steps'],
                                  save_steps = config['data']['steps'],
                                 learning_rate=float(config['model']['learning_rate']),
                                  warmup_steps=config['model']['num_epochs'])

In [ ]:
model.to(device)
trainer = Trainer(model=model, args=training_args,  train_dataset=train, eval_dataset=test, 
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 2023
  Num Epochs = 40
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 80920


Step,Training Loss
10000,0.722700
20000,0.136600
30000,0.040500
40000,0.021100
50000,0.013000
60000,0.009000
70000,0.006900
80000,0.005500


Saving model checkpoint to /content/drive/MyDrive/data/results/checkpoint-10000
Configuration saved in /content/drive/MyDrive/data/results/checkpoint-10000/config.json
Model weights saved in /content/drive/MyDrive/data/results/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/data/results/checkpoint-20000
Configuration saved in /content/drive/MyDrive/data/results/checkpoint-20000/config.json
Model weights saved in /content/drive/MyDrive/data/results/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/data/results/checkpoint-30000
Configuration saved in /content/drive/MyDrive/data/results/checkpoint-30000/config.json
Model weights saved in /content/drive/MyDrive/data/results/checkpoint-30000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/data/results/checkpoint-40000
Configuration saved in /content/drive/MyDrive/data/results/checkpoint-40000/config.json
Model weights saved in /content/drive/MyDrive/

TrainOutput(global_step=80920, training_loss=0.11811539083203643, metrics={'train_runtime': 12031.3312, 'train_samples_per_second': 6.726, 'train_steps_per_second': 6.726, 'total_flos': 2.836139727310848e+16, 'train_loss': 0.11811539083203643, 'epoch': 40.0})

In [ ]:
saved_root = config['data']['saved_root_internacional']

In [ ]:
trainer.save_model(saved_root)

Saving model checkpoint to /content/drive/MyDrive/data/small-gpt-neo-emo-bot-internacionais-100-epochs
Configuration saved in /content/drive/MyDrive/data/small-gpt-neo-emo-bot-internacionais-100-epochs/config.json
Model weights saved in /content/drive/MyDrive/data/small-gpt-neo-emo-bot-internacionais-100-epochs/pytorch_model.bin


In [ ]:
50000

50000

In [ ]:
model = GPTNeoForCausalLM.from_pretrained(saved_root)

loading configuration file /content/drive/MyDrive/data/small-gpt-neo-emo-bot-internacionais-100-epochs/config.json
Model config GPTNeoConfig {
  "_name_or_path": "EleutherAI/gpt-neo-125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 12,
  "num_layers": 12,
  "resid_dropout": 0,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "sum

In [ ]:
model.to('cuda')
generated = tokenizer('I miss you so much',return_tensors='pt').input_ids.cuda()

In [ ]:
#Generating texts
sample_outputs = model.generate(generated, 
                 # Use sampling instead of greedy decoding 
                 do_sample=True, 
                 # Keep only top 3 token with the highest probability
                 top_k=10, 
                 # Maximum sequence length
                 max_length=200, 
                 # Keep only the most probable tokens with cumulative probability of 95%
                 top_p=0.95, 
                 # Changes randomness of generated sequences
                 temperature=2.,
                 # Number of sequences to generate                 
                 num_return_sequences=3)
# Decoding and printing sequences
for i, sample_output in enumerate(sample_outputs):
    texto = tokenizer.decode(sample_output.tolist())
    regex_padding = re.sub('<|pad|>', '', texto)
    regex_barra = re.sub('[|+]', '', regex_padding)
    espaço = re.sub('[ +]', ' ', regex_barra)
    resultado = re.sub('[\n](2, )', '\n', espaço)
    print(">> Texto {}: {}".format(i+1, resultado))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


>> Texto 1: I miss you so much, you little miss me 
 I miss you so much, you little miss me 
 Oh no, I won't forget you, no 
 If you had my way, you would be out of place 
 If you don't mind, I need a place to hide 
 So if you have a friend, you should know that they somehow 
 Understand and you can make them perfect 
 I wish that you would just leave, so when the world ends 
 I wish that you would just take a second chance 
 To make them understand 
 And leave them all behind 
 I need a little room to breathe 
 To applause, I need a room to breathe 
 In the dark, I wish I could learn to love my scars 
 But I cannot catch fire from my burning soul 
 If you have a friend, you should know that they somehow 
 You don't mind, I know youll be here 
>> Texto 2: I miss you so much, uh, I'll miss you so much. 
 I know you miss me, uh, I'll miss you so much. 
 Don't wanna see you drop something 
 Don't wanna feel all alone 
 I wanna feel you touch, 
 Um every time you're around 
 I wanna feel y